<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END2_Phase1/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install transformers

In [7]:
!pip install pytorch-lightning

In [8]:
# Reading and data and display sample

import pandas as pd
import os
import torch
from transformers import BertTokenizer, BertModel
import time
import json
from pathlib import PurePath
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f'device is {device}')


device is cuda


In [9]:
pl.seed_everything(42)

Global seed set to 42


42

In [10]:
file_path = 'nikhil_onlytextpytorch.json'

path = os.path.join(os.getcwd(),file_path)

train = pd.read_json(path, orient='index').reset_index()
display(train)

,index,answer,question,context,source
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
1,"By default,NaN is replaced with what value?",zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
2,What values are used to replace negative infin...,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
3,What is the default value for negative infinity?,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
4,What is the value to replace positive infinity...,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN...",https://pytorch.org/docs/stable/generated/torc...
...,...,...,...,...,...
998,What is the tensor of shape where*is zero or m...,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...,https://pytorch.org/docs/stable/generated/torc...
999,What is the tolerance value to determine when ...,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...,https://pytorch.org/docs/stable/generated/torc...
1000,What is the default value of atorch.Tensor?,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...
1001,What computes the inverse of a square matrix?,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...


In [11]:
train = (train[['question', 'answer']])


In [12]:
display(train)

,question,answer
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf"
1,"By default,NaN is replaced with what value?",zero
2,What values are used to replace negative infin...,"bynan,posinf, andneginf"
3,What is the default value for negative infinity?,the least finite value
4,What is the value to replace positive infinity...,posinf
...,...,...
998,What is the tensor of shape where*is zero or m...,A(Tensor)
999,What is the tolerance value to determine when ...,rcond
1000,What is the default value of atorch.Tensor?,1e-15
1001,What computes the inverse of a square matrix?,torch.linalg.inv()


In [13]:
import torch

class TextTokenizer(torch.utils.data.Dataset):
    def __init__(
        self,
        data:pd.DataFrame, 
        tokenizer:BertTokenizer,
        column_name:str,
        source_max_length: int=30,
        ):
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_length = source_max_length
        self.column_name = column_name


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index:int):
        data_row = self.data.iloc[index]
        question_encoding = tokenizer(
            data_row['question'],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
        
        answer_encoding = tokenizer(
            data_row['answer'],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")


        return dict(
            input_ids_questions = question_encoding['input_ids'].flatten(),
            attention_mask_questions = question_encoding['attention_mask'].flatten(),
            input_ids_answers = answer_encoding['input_ids'].flatten(),
            attention_mask_answers = answer_encoding['attention_mask'].flatten()
        )


In [14]:

dataset = TextTokenizer(train, tokenizer, column_name='question')

trainloader = torch.utils.data.DataLoader(dataset, batch_size=8, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
next(iter(trainloader))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


{'attention_mask_answers': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]]),
 'attention_mask_questions': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [16]:
train_df, val_df = train_test_split(train, test_size=0.30)

In [17]:
class QnaLoaders(pl.LightningDataModule):

    def __init__(
        self,
        text_tokenizer:TextTokenizer,
        train_df:pd.DataFrame,
        val_df:pd.DataFrame,
        column_name:str,
        tokenizer=tokenizer,
        batch_size:int=8,
        max_token_length:int=30):

        super().__init__()
        self.batch_size=batch_size
        self.train_df = train_df
        self.val_df = val_df,
        self.tokenizer = tokenizer
        self.text_tokenizer = text_tokenizer
        self.max_token_length=max_token_length
        self.column_name = column_name


        
    def setup(self):
        self.train_dataset = self.text_tokenizer(
            self.train_df,
            self.tokenizer,
            self.column_name,
            self.max_token_length  
        )

        self.val_dataset = self.text_tokenizer(
            self.val_df,
            self.tokenizer,
            self.column_name,
            self.max_token_length  
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4
        )
    

In [18]:
BATCH_SIZE = 32
N_EPOCHS = 10

data_module = QnaLoaders(TextTokenizer,
                         train_df=train_df,
                         val_df=val_df,
                         tokenizer=tokenizer,
                         column_name='question',
                         batch_size=BATCH_SIZE,
                         max_token_length=30)

In [19]:
data_module.setup()


In [20]:
train_loader = data_module.train_dataloader()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Model - BERT-Pretrained

In [21]:
class EnsembleTokens(torch.nn.Module):
    def __init__(self):
        super(EnsembleTokens, self).__init__()
        self.question_model = BertModel.from_pretrained('bert-base-uncased')
        self.answer_model = BertModel.from_pretrained('bert-base-uncased')
    
    def forward(self, questions_encoding):

        question_outputs = self.question_model(questions_encoding['input_ids_questions'].to(device), questions_encoding['attention_mask_questions'].to(device))
        answer_outputs = self.question_model(questions_encoding['input_ids_answers'].to(device), questions_encoding['attention_mask_answers'].to(device))
        

        return question_outputs, answer_outputs

In [22]:
model = EnsembleTokens().to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

In [23]:
question_encoding, answer_encoding = model(next(iter(train_loader)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [24]:
question_encoding[0]

tensor([[[-0.4242, -0.0142, -0.0425,  ..., -0.4050,  0.1820,  0.6914],
         [-0.1860, -0.3973, -0.3700,  ...,  0.0553,  0.1730,  0.4024],
         [-0.2162, -0.4117,  0.1014,  ..., -0.2265, -0.2873,  0.8317],
         ...,
         [-0.1354, -0.2090, -0.1617,  ...,  0.1289,  0.0898,  0.1525],
         [-0.3192, -0.1442,  0.1922,  ..., -0.2208, -0.0639,  0.5473],
         [-0.3068, -0.2415,  0.1832,  ..., -0.2501, -0.0296,  0.5294]],

        [[-0.4400, -0.4635,  0.1354,  ..., -0.1976,  0.1560,  0.9777],
         [-0.2591, -0.8249, -0.2640,  ...,  0.1651,  0.1408,  0.3448],
         [-0.3390, -0.5018,  0.6050,  ..., -0.2229, -0.8399,  0.6102],
         ...,
         [-0.0467, -0.6815,  0.1278,  ...,  0.1198, -0.1004,  0.0422],
         [ 0.0039, -0.6308,  0.5286,  ..., -0.0202, -0.1765,  0.2662],
         [-0.1485, -0.7141, -0.0385,  ...,  0.0642, -0.0954,  0.0099]],

        [[-0.3174, -0.2040,  0.0096,  ..., -0.4341,  0.3590,  0.5546],
         [-0.1596, -0.4356, -0.2471,  ...,  0

In [25]:
list(model.children())[0]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [26]:
# for param in list(model.children())[0]:
#     param.requires_grad = False


TypeError: ignored

In [27]:
from torch.optim import AdamW

criterion = torch.nn.CosineSimilarity()

learning_rate = 5e-5
adam_epsilon=1e-8
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

In [35]:

num_train_epochs = 10
fraction_per_epoch = 0.2

print("***** Running training *****")
print("  Num examples = %d" % len(dataset))
print("  Num Epochs = %d" % num_train_epochs)
print("  Batch size = %d" % BATCH_SIZE)
print("  Total optimization steps = %d" % (len(train_loader) // (num_train_epochs * fraction_per_epoch)))

model.zero_grad()
train_iterator = trange(num_train_epochs, desc="Epoch")

train_loss_set = []
loss = 0
for _ in train_iterator:
    print(loss)
    epoch_iterator = tqdm(iter(train_loader), desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
    #   if step < global_step + 1:
    #     continue

      model.train()
    #   batch = tuple(t.to(device) for t in batch)

      question_encoding, answer_encoding = model(batch)
      loss = 1. - criterion(question_encoding[0], answer_encoding[0])
      train_loss_set.append(loss)
      print(f'####$$$$$$$%%%%%% Loss  {loss.mean()}')
    #   print(loss, loss.shape)
      loss = loss.mean().backward()
      
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    #   tr_loss += loss.item()
      optimizer.step()
      model.zero_grad()
    #   global_step += 1
    
    #   if global_step % 1000 == 0:
    #     print("Train loss: {}".format(tr_loss/global_step))
    #     output_dir = 'checkpoint-{}'.format(global_step)
    #     if not os.path.exists(output_dir):
    #         os.makedirs(output_dir)
    #     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    #     model_to_save.save_pretrained(output_dir)
    #     torch.save(torch.tensor(train_loss_set), os.path.join(output_dir, 'training_loss.pt'))
    #     print("Saving model checkpoint to %s" % output_dir)

***** Running training *****
  Num examples = 1003
  Num Epochs = 10
  Batch size = 32
  Total optimization steps = 11


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7069074200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7069074200>
Traceback (most recent call last):
  File "/usr/local/lib/python3

####$$$$$$$%%%%%% Loss  0.01996014080941677


...


Iteration:  91%|█████████ | 20/22 [00:15<00:01,  1.31it/s]

####$$$$$$$%%%%%% Loss  0.009321140125393867



Iteration:  95%|█████████▌| 21/22 [00:16<00:00,  1.32it/s]

####$$$$$$$%%%%%% Loss  0.009355925023555756



Epoch: 100%|██████████| 10/10 [02:52<00:00, 17.20s/it]
